 ## EDA 

#### 1. __Cargar librerías y datos__


In [ ]:
# Data analysis and wrangling
import numpy as np
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Warnings configuration
import warnings
warnings.filterwarnings('ignore')


Cargamos el conjunto de datos desde el archivo CSV


In [ ]:
df = pd.read_csv("../data/attrition_availabledata_03.csv")

#### 2. __Exploración inicial__

En esta sección revisamos la estructura general del dataset


In [ ]:
print(df.info())

In [ ]:
dataset_shape = df.shape
print(f"El dataset contiene {dataset_shape[0]} filas y {dataset_shape[1]} columnas.")

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df[['Attrition']].head()

Este es un problema de __clasificación__, ya que la variable objetivo (Attrition) es binaria (Yes / No). Esto significa que el modelo debe predecir si un empleado abandonará o no la empresa, en lugar de predecir un valor numérico.

#### 3. __Identificamos las variables categóricas y numéricas__

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
print("Variables categóricas:", categorical_columns)
print("Variables numéricas:", numerical_columns)

- Reclasificamos las variables añadiendo ordinales

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

# Eliminamos de numéricas las que hemos clasificado como ordinales
numerical_columns = [col for col in numerical_columns if col not in ordinal_columns]

print("Variables categóricas:", categorical_columns)
print("Variables ordinales:", ordinal_columns)
print("Variables numéricas:", numerical_columns)


- Detección de Variables Categóricas con Alta Cardinalidad

 Identificamos variables categóricas que pueden generar demasiadas columnas al codificarlas


In [ ]:
categorical_cardinality = df[categorical_columns].nunique().sort_values(ascending=False)
display(categorical_cardinality)


No consideramos que existan variables categóricas de alta cardinalidad, por lo tanto no necesitaremos realizar ninguna agrupación adicional o una diferente codificación.

#### 4. __Análisis de la variable objetivo__

Revisamos la distribución de la variable objetivo para identificar desbalance de clases


In [ ]:
if "Attrition" in df.columns:
    plt.figure(figsize=(4,4))
    sns.countplot(x=df["Attrition"], palette="viridis")
    plt.title("Distribución de la variable objetivo (Attrition)\n")
    plt.show()
    
    attrition_counts = df["Attrition"].value_counts(normalize=True)
    display(pd.DataFrame(attrition_counts).rename(columns={"Attrition": "Proportion"}).reset_index(drop=True)*100)


In [ ]:
df.Attrition.value_counts().sort_index().to_frame()

 El dataset está desbalanceado, con 2466 empleados que no abandonan la empresa (NO) y 474 que sí lo hacen (SI). 
 
 Esto significa que la mayoría de los empleados no abandonan la empresa, lo que podría causar que un modelo mal entrenado siempre prediga "No", logrando una precisión aparente alta, pero sin realmente capturar los casos de abandono.

Por lo tanto la solución sería aplicar técnicas como:

- Oversampling (SMOTE): Aumentar el número de ejemplos de la clase minoritaria (Yes).
- Undersampling: Reducir los casos de la clase mayoritaria (No).
- Pesos en los modelos: Ajustar la importancia de la clase minoritaria. 



#### 5. __Identificar valores nulos__

In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].to_frame().reset_index()
missing_values.columns = ["Column Name", "Missing Values"]

display(missing_values)

Ya que los valores faltantes son muy pocos en comparación con el número de datos totales vamos a tomar las siguientes medidas:
- Imputamos valores faltantes en las variables numéricas con la mediana
- Imputamos valores faltantes en las variables categóricas con la moda

In [ ]:
for col in df.select_dtypes(include=['number']).columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

print("Valores nulos imputados correctamente.")

#### 6. __Identificar variables constantes e identidicativas__

- Comprobamos si existen columnas con valores únicos

In [ ]:
unique_values = df.nunique()

constant_columns = df.nunique()[df.nunique() == 1].to_frame().reset_index()
constant_columns.columns = ["Column Name", "Unique Value Count"]
constant_columns["Unique Value"] = constant_columns["Column Name"].apply(lambda col: df[col].unique()[0])
constant_columns_list = constant_columns["Column Name"].tolist()

display(constant_columns)

- Comprobamos si hay columnas con variables ID

In [ ]:
print(f"Número total de filas: {len(df)}")
print(f"Número de valores únicos en EmployeeID: {df['EmployeeID'].nunique()}")

id_columns = [col for col in df.columns if df[col].nunique() == len(df)]
print("Columnas identificativas detectadas:", id_columns)


- Eliminar las columnas constantes e identificativas

In [ ]:
df = df.drop(columns=constant_columns_list + id_columns, errors='ignore')
numerical_columns = [col for col in numerical_columns if col not in (constant_columns_list + id_columns)]

print('Se han eliminado las columnas:', constant_columns_list + id_columns)

#### 7. __Crear matriz de correlación__

Generamos la matriz de correlación para entender relaciones entre las variables numéricas, habiendo eliminado ya las constantes y las identificativas.

In [ ]:
df_numeric = df.select_dtypes(include=['number'])

correlation_matrix = df_numeric.corr()


plt.figure(figsize=(18, 18))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", linewidths=0.5, cmap="twilight")
plt.xticks(rotation=75)
plt.title("Matriz de Correlación\n")
plt.show()

No hay correlaciones extremadamente altas, cercanas a 1 o -1, las variables no son redundantes ni son fuertemente dependientes entre sí. Pero sí algunas correlaciones moderadas que podemos considerar.

__Relación entre antigüedad y experiencia laboral:__

- YearsAtCompany y YearsWithCurrManager (0.76): Los empleados que llevan más tiempo en la empresa, más probabilidad de que hayan estado más tiempo con el mismo gerente.
- YearsAtCompany y TotalWorkingYears (0.62): Cuanto más tiempo ha trabajado una persona en general, más tiempo puede haber pasado en la empresa actual.

__Relación entre PercentSalaryHike y PerformanceRating (0.78):__ Hay una alta correlación entre el incremento salarial y la calificación de desempeño. Los empleados con mejor rendimiento reciben mayores aumentos salariales.

- Podríamos evaluar si PercentSalaryHike es redundante, ya que está fuertemente ligada a PerformanceRating.



En cuanto al resto, tienen correlaciones muy bajas con todas las demás, lo que indica que pueden ser independientes o estar influenciadas por otros factores no considerados. Así, podríamos revisar si estas variables tienen algún impacto significativo en la variable objetivo, o si pueden eliminarse.

#### 8. __Identificamos la correlación con la variable objetivo__

In [ ]:
df["Attrition"] = df["Attrition"].map({"Yes": 1, "No": 0})

# Asegurar que solo trabajamos con columnas numéricas
df_corr = df.select_dtypes(include=['number'])
attrition_correlation = df_corr.corr()["Attrition"].sort_values()

plt.figure(figsize=(10, 6))
ax = sns.barplot(y=attrition_correlation.index, x=attrition_correlation.values, palette="coolwarm")

# Añadir valores en las barras
for index, value in enumerate(attrition_correlation.values):
    ax.text(value, index, f"{value:.2f}", ha="left", va="center", fontsize=10, color="black")

plt.title("Correlación de 'Attrition' con el resto de variables")
plt.xlabel("Correlación")
plt.ylabel("Variables")
plt.show()


Podemos comprobar que no hay una variable con una correlación extremadamente fuerte con Attrition, lo que indica que el abandono es un fenómeno multifactorial, es decir todas las variables afectan en algo a la variable objetivo.

Las variables de antigüedad (YearsAtCompany, TotalWorkingYears) son las que más influyen en la retención, podemos evaluar las relaciones con YearsWithCurrManager para comprobar si esta última sería redundante.

#### __9. Visualizamos relaciones entre las variables correlacionadas__
Realizamos una exploración visual de las relaciones entre las variables con alta correlación, para ayudarnos a decidir si hay redundancias o si algunas variables debemos transformarlas antes de usarlas en un modelo predictivo.

In [ ]:

# Visualizar relaciones entre variables de antigüedad con pairplot
sns.pairplot(df, vars=["YearsAtCompany", "YearsWithCurrManager", "TotalWorkingYears"], diag_kind="kde")
plt.suptitle("Relaciones entre Antigüedad y Experiencia Laboral", y=1.02)
plt.show()



__Relación entre Antigüedad y Experiencia Laboral__

- YearsAtCompany y YearsWithCurrManager (0.76): Relación fuerte, posible redundancia.
- YearsAtCompany y TotalWorkingYears (0.62): Relación esperada, pero aporta información diferente.
- Conclusión: YearsAtCompany o YearsWithCurrManager podría ser redundante.

Se podría evaluar impacto en el modelo y eliminar una si es necesario.

In [ ]:
# Visualizar relación entre incremento salarial y evaluación de desempeño
sns.pairplot(df, vars=["PercentSalaryHike", "PerformanceRating"], diag_kind="kde")
plt.suptitle("Relación entre Aumento Salarial y Evaluación de Desempeño", y=1.02)
plt.show()



__Relación entre Aumento Salarial y Evaluación de Desempeño__

- PerformanceRating tiene pocos valores (3.0 y 4.0), indicando que es categórica y poco variable.
- PercentSalaryHike tiene más variabilidad y está correlacionada con PerformanceRating (0.78).
- Conclusión: PerformanceRating aporta poca información adicional y podría eliminarse.

Se podría considerar eliminarla o transformarla en variable binaria (alta/baja).